In [ ]:
# You may need to install:
# !pip install git+https://github.com/vafaei-ar/ccgpack.git

In [ ]:
import os
import sys
import shutil
import argparse
import numpy as np
import pylab as plt
import healpy as hp
from ccgpack import sky2patch,download,ch_mkdir,pop_percent
from utils import *

#from healpy import cartview

cmap = plt.cm.jet
cmap.set_under('w')
cmap.set_bad('gray',1.)

if not in_notebook():
    import argparse
    parser = argparse.ArgumentParser(description='Short sample app')
    parser.add_argument('-r', action="store_true", default=False)
    parser.add_argument('--nsim', action="store", type=int, default=42)
    args = parser.parse_args()
    replace = args.r
    nsim = args.nsim
else:
    replace = 0
    nsim = 10
nside = 2048
lmax = 2*nside

You can download CMB end2end simulations from:
https://wiki.cosmos.esa.int/planck-legacy-archive/index.php/CMB_maps#CMB%20simulations

# download template:
-     http://pla.esac.esa.int/pla/aio/dx12_v3_{method}_{cmb,noise,noise_hm1,noise_hm2,noise_oe1,noise_oe2}_mc_?????_raw.fits

In [ ]:
!mkdir -p maps
!mkdir -p maps/dataset

In [ ]:
for idx in range(nsim):
    !eval {"wget -q http://pla.esac.esa.int/pla/aio/product-action?SIMULATED_MAP.FILE_ID=dx12_v3_sevem_cmb_mc_{:05d}_raw.fits -O maps/dx12_v3_sevem_cmb_mc_{:05d}_raw.fits".format(idx,idx)}
    !eval {"wget -q http://pla.esac.esa.int/pla/aio/product-action?SIMULATED_MAP.FILE_ID=dx12_v3_sevem_noise_mc_{:05d}_raw.fits -O maps/dx12_v3_sevem_noise_mc_{:05d}_raw.fits".format(idx,idx)}

In [ ]:
path_temp = 'maps/dx12_v3_{}_{}_mc_{:05d}_raw.fits'
csep = 'sevem'
typ = 'cmb'

for iii in range(nsim):
    for csep in ['sevem']:#,'nilc','smica','commander']:
        fname1 = path_temp.format(csep,'cmb',iii)
        fname2 = path_temp.format(csep,'noise',iii)
        m = hp.read_map(fname1,nest=1,verbose=0)+hp.read_map(fname2,nest=1,verbose=0)
        patches = sky2patch(m,1)
        for j in range(12):
            fnsave = 'maps/dataset/{}-{}-{}'.format(csep,iii,j)
            np.save(fnsave,patches[j])
            plt.imshow(patches[j], cmap=cmap)
            plt.tight_layout()
            plt.savefig(fnsave+'.jpg')
            plt.close()

cl = np.load('cl_planck_lensed.npy')
ll = cl[:lmax,0]
cl = cl[:lmax,1]
ll = np.concatenate([[0,1],ll])
cl = np.concatenate([[0,0],cl])

for iii in range(nsim):
    m = hp.sphtfunc.synfast(cl, nside=nside, lmax=lmax, mmax=None, alm=0, pol=0, pixwin=0, fwhm=0, sigma=None, new=1, verbose=0)
    m = hp.reorder(m , r2n=1)
    patches = sky2patch(m,1)
    for j in range(12):
        fnsave = 'maps/dataset/{}-{}-{}'.format('healpix',iii,j)
        np.save(fnsave,patches[j])
        plt.imshow(patches[j], cmap=cmap)
        plt.tight_layout()
        plt.savefig(fnsave+'.jpg')
        plt.close()

In [ ]:
for csep in ['healpix','sevem']:#,'nilc','smica','commander']:
    print(csep)
    pps = []
    for iii in range(nsim):
        for j in range(12):
            fnsave = 'maps/dataset/{}-{}-{}.npy'.format(csep,iii,j)
            pp = np.load(fnsave)
            pps.append(pp)
    pps = np.array(pps)
    np.save(csep,pps)